In [8]:
import csv
import numpy as np

with open('../.idea/po_słowach_integers-po_slowach-matrix-pl-20211020', 'r', encoding='UTF-8') as file:
    matrix_raw = file.readlines()
    del matrix_raw[0]
    matrix = []
    article_id = []
    for row in matrix_raw:
        splitted_row = row.split()
        article_id.append(splitted_row[0])
        del splitted_row[0]
        matrix.append(splitted_row)

with open('../.idea/po_słowach_integers-po_slowach-cats_dict-pl-20211020', 'r', encoding='UTF-8') as file:
    data_iter = csv.reader(file, delimiter = '\t')
    print(data_iter)
    data = [data for data in data_iter]
    print(data_iter)
categories_dict = {int(i[1]):i[0] for i in data}
print(categories_dict)

with open('../.idea/po_słowach_integers-po_slowach-categories-pl-20211020', 'r', encoding='UTF-8') as file:
    data_iter = csv.reader(file, delimiter = '\t')
    data = [data for data in data_iter]
categories_articles = np.asarray(data, dtype=np.int32)

with open('../.idea/po_słowach_integers-po_slowach-articles_dict-pl-20211020', 'r', encoding='UTF-8') as file:
    data_iter = csv.reader(file, delimiter = '\t')
    data = [data for data in data_iter]
articles_dict = {int(i[1]):i[0] for i in data}

{5046178: 'Bułgarscy_satyrycy', 3889979: 'Obchody_rocznicowe', 3547238: 'Polskie_filmy_o_podróżach_w_czasie', 1698632: 'Radiestezja', 139508: 'Rozrywki_umysłowe', 3752700: 'Upamiętnienie_Świętego_Wojciecha', 4721243: 'Akcesoria_do_palenia', 842670: 'Naukowe_projekty_meteorologiczne', 3565288: 'Albumy_Janet_Jackson', 3941982: 'Motocykle_turystyczne'}


In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
X = np.array(matrix, dtype=np.int32)
y = categories_articles[:, 1]
X_train, X_test, y_train, y_test = train_test_split(X, y)



In [3]:
#SVM (LinearSVC)

from sklearn.svm import LinearSVC
m_LinearSVC = LinearSVC(tol=1.0e-6,max_iter=5000,verbose=1)
#m_LinearSVC.fit(X_train, y_train)
#results = m_LinearSVC.predict(X_test)
scores = cross_val_score(m_LinearSVC, X, y, cv=5, scoring='recall_macro')
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][0.95 0.95 1.   1.   0.9 ]
Accuracy: 0.96 (+/- 0.04)


In [4]:
#KNN

from sklearn.neighbors import KNeighborsClassifier
m_KNN = KNeighborsClassifier()
scores = cross_val_score(m_KNN, X, y, cv=5, scoring='recall_macro')
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

[0.7  0.45 0.55 0.55 0.45]
Accuracy: 0.54 (+/- 0.09)


In [5]:
#Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
m_MNB = MultinomialNB()
scores = cross_val_score(m_MNB, X, y, cv=5, scoring='recall_macro')
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

[1.   1.   0.95 1.   0.85]
Accuracy: 0.96 (+/- 0.06)


In [6]:
#Multi-layer Perceptron
from sklearn.neural_network import MLPClassifier
m_MLP = MLPClassifier()
scores = cross_val_score(m_MLP, X, y, cv=5, scoring='recall_macro')
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

[1.   1.   0.95 1.   0.85]
Accuracy: 0.96 (+/- 0.06)


In [7]:
# Przygotowanie bag of words
from numpy import genfromtxt, int32
my_data = genfromtxt('../.idea/po_słowach_integers-po_slowach-matrix-pl-20211020', dtype = int32, skip_header = 1, delimiter=' ')
bag_of_words = dict()

for row in my_data:
    row_dict = dict()
    for i, word_count in enumerate(row[1:], start = 1):
        if word_count > 0:
            row_dict[i] = word_count
    bag_of_words[row[0]] = row_dict
